In [1]:
%cd ..

/home/chanwutk/code/apperception


In [2]:
import json
import os
import pickle
import traceback

import numpy as np

from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline

# Stages
from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway

from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame

from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_2d.object_type_filter import ObjectTypeFilter
from optimized_ingestion.stages.detection_2d.ground_truth import GroundTruthDetection

from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad as FromD2DAndRoad

from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.detection_estimation.segment_mapping import RoadPolygonInfo

from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
from optimized_ingestion.stages.tracking_2d.tracking_2d import Tracking2D, Tracking2DResult

from optimized_ingestion.stages.tracking_3d.from_2d_and_road import From2DAndRoad as FromT2DAndRoad
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult, Tracking3D

from optimized_ingestion.stages.segment_trajectory import SegmentTrajectory
from optimized_ingestion.stages.segment_trajectory.construct_segment_trajectory import SegmentPoint
from optimized_ingestion.stages.segment_trajectory.from_tracking_3d import FromTracking3D

from optimized_ingestion.stages.get_camera_config_id import GetCameraConfigId

# Actions
from optimized_ingestion.actions.bbox2d_overlay import bbox2d_overlay

from optimized_ingestion.video import Video

/home/chanwutk/.installs/mambaforge/envs/apperception/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5 🚀 2023-3-12 Python-3.10.9 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14966MiB)

Using cache found in /home/chanwutk/code/apperception/weights/ultralytics_yolov5_master


Using cuda:0


YOLOv5 🚀 2023-3-12 Python-3.10.9 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14966MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
from optimized_ingestion.cache import disable_cache
disable_cache()

In [4]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [5]:
import torch

In [6]:
import shapely.wkt

class DataclassJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Tracking3DResult):
            return {
                "frame_idx": o.frame_idx,
                "detection_id": tuple(o.detection_id),
                "object_id": o.object_id,
                "point_from_camera": o.point_from_camera,
                "point": o.point.tolist(),
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "timestamp": str(o.timestamp),
            }
        if isinstance(o, Tracking2DResult):
            return {
                "detection_id": tuple(o.detection_id),
                "frame_idx": o.frame_idx,
                "object_id": o.object_id,
                "bbox_left": o.bbox_left,
                "bbox_top": o.bbox_top,
                "bbox_w": o.bbox_w,
                "bbox_h": o.bbox_h,
                "object_type": o.object_type,
                "confidence": o.confidence
            }
        if isinstance(o, SegmentPoint):
            return {
                "detection_id": tuple(o.detection_id),
                "car_loc3d": o.car_loc3d,
                "timestamp": str(o.timestamp),
                "segment_line": None if o.segment_line is None else o.segment_line.to_ewkb(),
#                 "segment_line_wkb": o.segment_line.wkb_hex,
                "segment_heading": o.segment_heading,
                "road_polygon_info": o.road_polygon_info,
                "obj_id": o.obj_id,
                "type": o.type,
                "next": None if o.next is None else tuple(o.next.detection_id),
                "prev": None if o.prev is None else tuple(o.prev.detection_id),
                
            }
        if isinstance(o, RoadPolygonInfo):
            return {
                "id": o.id,
                "polygon": str(o.polygon),
#                 "polygon_wkb": o.polygon.wkb_hex,
                "segment_lines": [str(l) for l in o.segment_lines],
                "road_type": o.road_type,
                "segment_headings": o.segment_headings,
                "contains_ego": o.contains_ego,
                "ego_config": o.ego_config,
                "fov_lines": o.fov_lines
            }
        if isinstance(o, torch.Tensor):
            return o.tolist()
        if isinstance(o, np.ndarray):
            return o.tolist()
        return super().default(o)

In [7]:
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

True
/home/chanwutk/data/processed/full-dataset/mini


In [8]:
if NUSCENES_PROCESSED_DATA in os.environ:
    DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
else:
    DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
with open(os.path.join(DATA_DIR, "videos", "frames.pkl"), "rb") as f:
    videos = pickle.load(f)

In [9]:
def run_benchmark(pipeline, filename, ignore_error=False):
    metadata_strongsort = {}
    metadata_3d = {}
    metadata_segment = {}
    metadata_frame_id = {}
    failed_videos = []
    for i, (name, video) in enumerate(videos.items()):
        try:
            print(name, f'---{i} / {len(videos)}---------------------------------------------------------')
            video_filename = video['filename']
            print(video_filename)
            if not video_filename.startswith('boston') or 'FRONT' not in name:
                continue
                
            for v in video['frames'][:10]:
                print(v[1])
            frames = Video(
                os.path.join(DATA_DIR, "videos", video["filename"]),
                [camera_config(*f, 0) for f in video["frames"]],
            )

            output = pipeline.run(Payload(frames))
#             metadata_strongsort[name] = output[StringSORT]
#             metadata_3d[name] = output[Tracking3D]
#             metadata_segment[name] = output[SegmentTrajectory]
#             metadata_frame_id[name] = output[GetCameraConfigId]
            
            bbox2d_overlay(output, './outputs/bbox2d_overlay')
            
            
        except Exception as e:
            if ignore_error:
                failed_videos.append((name, str(traceback.format_exc())))
                print('ERRORRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR')
                print('ERRORRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR')
                print('ERRORRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR')
            else:
                raise e

#         # Save progress every video
#         with open(f"./outputs/{filename}.json", "w") as f:
#             json.dump(metadata_strongsort, f, cls=DataclassJSONEncoder, indent=2)
            
#         with open(f"./outputs/{filename}-frame-id.json", "w") as f:
#             json.dump(metadata_frame_id, f, indent=2)

#         with open(f"./outputs/{filename}-3d.json", "w") as f:
#             json.dump(metadata_3d, f, cls=DataclassJSONEncoder, indent=2)

#         with open(f"./outputs/{filename}-segment.json", "w") as f:
#             json.dump(metadata_segment, f, cls=DataclassJSONEncoder, indent=2)

#         with open(f"./outputs/{filename}-failed-videos.json", "w") as f:
#             json.dump(failed_videos, f, indent=2)

#         with open(f"./outputs/{filename}-performance.json", "w") as f:
#             performance = [
#                 {
#                     "stage": stage.classname(),
#                     "runtimes": stage.runtimes,
#                 }
#                 for stage
#                 in pipeline.stages
#             ]
#             json.dump(performance, f, indent=2)

In [10]:
type_filter = ['bicycle', 'motorcycle', 'car', 'truck', 'bus']

In [11]:
# pipeline1 = Pipeline()
# # pipeline1.add_filter(ParallelDecodeFrame())
# pipeline1.add_filter(DecodeFrame())
# pipeline1.add_filter(filter=YoloDetection())
# pipeline1.add_filter(filter=ObjectTypeFilter(type_filter))

# pipeline1.add_filter(filter=FromD2DAndRoad())
# pipeline1.add_filter(filter=StrongSORT(cache=True))
# # pipeline1.add_filter(filter=StrongSORT(cache=False))
# pipeline1.add_filter(filter=FromT2DAndRoad())

# pipeline1.add_filter(filter=FromTracking3D())
# pipeline1.add_filter(filter=GetCameraConfigId())

# run_benchmark(pipeline1, 'segment-tracking-without-de', ignore_error=True)

In [12]:
# from optimized_ingestion.stages.depth_estimation import DepthEstimation
# from optimized_ingestion.stages.tracking_3d.from_2d_and_depth import From2DAndDepth

# pipeline3 = Pipeline()
# pipeline3.add_filter(ParallelDecodeFrame())
# # pipeline3.add_filter(DecodeFrame())
# pipeline3.add_filter(filter=YoloDetection())
# pipeline3.add_filter(filter=ObjectTypeFilter(type_filter))

# pipeline3.add_filter(filter=FromD2DAndRoad())
# pipeline3.add_filter(filter=StrongSORT(cache=True))
# pipeline3.add_filter(filter=DepthEstimation())
# pipeline3.add_filter(filter=From2DAndDepth())
# pipeline3.add_filter(filter=GetCameraConfigId())


# run_benchmark(pipeline3, 'tracking-with-depth-estimation', ignore_error=True)

In [13]:
# pipeline2 = Pipeline()
# # pipeline2.add_filter(ParallelDecodeFrame())
# pipeline2.add_filter(DecodeFrame())
# pipeline2.add_filter(filter=YoloDetection())
# pipeline2.add_filter(filter=ObjectTypeFilter(type_filter))

# pipeline2.add_filter(filter=FromD2DAndRoad())
# pipeline2.add_filter(filter=DetectionEstimation())
# pipeline2.add_filter(filter=StrongSORT())
# pipeline2.add_filter(filter=FromT2DAndRoad())

# pipeline2.add_filter(filter=SegmentTrajectory())
# pipeline2.add_filter(filter=GetCameraConfigId())

# # run_benchmark(pipeline2, 'segment-tracking-with-de')

In [14]:
with open(os.path.join(DATA_DIR, 'annotation_partitioned.pkl'), 'rb') as f:
    annotations = pickle.load(f)

pipeline4 = Pipeline()
pipeline4.add_filter(DecodeFrame())
pipeline4.add_filter(filter=GroundTruthDetection(annotations))
# pipeline4.add_filter(filter=ObjectTypeFilter(type_filter))

# pipeline4.add_filter(filter=FromD2DAndRoad())
# pipeline4.add_filter(filter=StrongSORT(cache=True))
# # pipeline4.add_filter(filter=StrongSORT(cache=False))
# pipeline4.add_filter(filter=FromT2DAndRoad())

# pipeline4.add_filter(filter=FromTracking3D())
# pipeline4.add_filter(filter=GetCameraConfigId())

run_benchmark(pipeline4, 'segment-tracking-gt-detection', ignore_error=False)

scene-0061-CAM_FRONT ---0 / 60---------------------------------------------------------
singapore-onenorth-scene-0061-CAM_FRONT.mp4
scene-0061-CAM_FRONT_RIGHT ---1 / 60---------------------------------------------------------
singapore-onenorth-scene-0061-CAM_FRONT_RIGHT.mp4
scene-0061-CAM_BACK_RIGHT ---2 / 60---------------------------------------------------------
singapore-onenorth-scene-0061-CAM_BACK_RIGHT.mp4
scene-0061-CAM_BACK ---3 / 60---------------------------------------------------------
singapore-onenorth-scene-0061-CAM_BACK.mp4
scene-0061-CAM_BACK_LEFT ---4 / 60---------------------------------------------------------
singapore-onenorth-scene-0061-CAM_BACK_LEFT.mp4
scene-0061-CAM_FRONT_LEFT ---5 / 60---------------------------------------------------------
singapore-onenorth-scene-0061-CAM_FRONT_LEFT.mp4
scene-0103-CAM_FRONT ---6 / 60---------------------------------------------------------
boston-seaport-scene-0103-CAM_FRONT.mp4
4f5e35aa6c6a426ca945e206fb2f4921
728e987b1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:00<00:00, 261.43it/s]


None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.GroundTruthDetection
None
229
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKK
dir ./outputs/bbox2d_overlay/boston-seaport-scene-0103-CAM_FRONT.mp4
scene-0103-CAM_FRONT_RIGHT ---7 / 60---------------------------------------------------------
boston-seaport-scene-0103-CAM_FRONT_RIGHT.mp4
5ed84fb1dbe24efcb00eb766a22d69d6
e12e484c47a14e1c979fcf2a983de41a
80c0e0c111b641b18b90954252c43d57
8fc862f04bb14f34aa408ba0ab92bd05
1a033e47f1374f64a01fa414bc8f0570
fe644fe101eb46fe86196f1af6ba26bb
55f48b2af8cd469a9d

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 233/233 [00:00<00:00, 276.92it/s]


None
233
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.GroundTruthDetection
None
233
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
dir ./outputs/bbox2d_overlay/boston-seaport-scene-0103-CAM_FRONT_RIGHT.mp4
scene-0103-CAM_BACK_RIGHT ---8 / 60---------------------------------------------------------
boston-seaport-scene-0103-CAM_BACK_RIGHT.mp4
scene-0103-CAM_BACK ---9 / 60---------------------------------------------------------
boston-seaport-scene-0103-CAM_BACK.mp4
scene-0103-CAM_BACK_LEFT ---10 / 60-------------------------------------------

...

None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.GroundTruthDetection
None
237
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
dir ./outputs/bbox2d_overlay/boston-seaport-scene-0757-CAM_FRONT.mp4
scene-0757-CAM_FRONT_RIGHT ---25 / 60---------------------------------------------------------
boston-seaport-scene-0757-CAM_FRONT_RIGHT.mp4
920a8ba176bb4197b2aebaeb8aa51ee1
14fe41675bf54c86a27c3fc7504b7929
8f07855567404b9484265b2ba90073f4
befcea3c350842d58300f604e2aa66b9
20a567382fea47a8a4be0a4533a620d5
cdd72eea4d3f4d93b28ce64d3b62692e
8

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 236/236 [00:00<00:00, 430.99it/s]


None
236
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.GroundTruthDetection
None
236
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
dir ./outputs/bbox2d_overlay/boston-seaport-scene-0757-CAM_FRONT_RIGHT.mp4
scene-0757-CAM_BACK_RIGHT ---26 / 60---------------------------------------------------------
boston-seaport-scene-0757-CAM_BACK_RIGHT.mp4
scene-0757-CAM_BACK ---27 / 60---------------------------------------------------------
boston-seaport-scene-0757-CAM_BACK.mp4
scene-0757-CAM_BACK_LEFT ---28 / 60-----------------------------------

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 236/236 [00:00<00:00, 443.55it/s]


None
236
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
Stage:  Detection2D.GroundTruthDetection
None
236
  filtered frames: 100.0%
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
dir ./outputs/bbox2d_overlay/boston-seaport-scene-0757-CAM_FRONT_LEFT.mp4
scene-0796-CAM_FRONT ---30 / 60---------------------------------------------------------
singapore-queenstown-scene-0796-CAM_FRONT.mp4
scene-0796-CAM_FRONT_RIGHT ---31 / 60---------------------------------------------------------
singapore-queenstown-scene-0796-CAM_FRONT_RIGHT.mp4
scene-0796-CAM_BACK_RIGHT ---32 / 60-------------------

In [15]:
with open(f"./outputs/segment-tracking-without-de-performance.json", "r") as f:
    benchmark = json.load(f)

ss_cache, ss_no_cache, *_ = benchmark[4:]
benchmark_data = [
    {
        'name': ssc['name'],
        'runtime_cache': ssc['runtime'],
        'runtime_no_cache': ssnc['runtime']
    }
    for ssc, ssnc
    in zip(ss_cache['runtimes'], ss_no_cache['runtimes'])
]
benchmark_data

FileNotFoundError: [Errno 2] No such file or directory: './outputs/segment-tracking-without-de-performance.json'

In [ ]:
import altair as alt
import pandas as pd

In [ ]:
(alt.Chart(pd.DataFrame.from_dict(benchmark_data))
    .mark_point()
    .encode(
        x='runtime_cache:Q',
        y='runtime_no_cache:Q',
        tooltip=['name']
    )
)